In [21]:
# use natural language toolkit
import nltk
from nltk.stem.lancaster import LancasterStemmer
# word stemmer
stemmer = LancasterStemmer()

In [2]:
import pandas as pd

In [3]:
sample=pd.DataFrame()

In [4]:
sample_data = []
sample_file = open("dataset.txt", encoding="utf8")
for line in sample_file:
    try:
        sample_data.append(line)
    except:
        continue
sample_data

['#token\tcategory\n',
 'I must look like a right weirdo .\tnon-intent\n',
 "I really don't want to get up and get dressed for work\ttrifle\n",
 'I like to watch golf #ibreakstereotypes\tevent\n',
 "I really want a hot dog . My co-workers went to go have them for lunch today . I'm jealous !\tfood\n",
 "I still haven't gotten what I want , and if I got it , I definitely know I'd want to keep it !\tnon-intent\n",
 'I need to reread the complete works of Conan Doyle .\tevent\n',
 'I wanna go to the beach , but ... Where is the people ? They never take the phone T.T\ttravel\n',
 'I guess ill go take a shower . hahaha , I dont really know what im suppose to be writting on this thing so , yeah . aha .\ttrifle\n',
 'I need to wake up . I need to wake up . I need to wake up . I HATE BEING A WOMAN .\ttrifle\n',
 'I am going to beat him over the head with a metal chair until he gets it ....\ttrifle\n',
 "I can now say i've had an obsession with cute clips since i was like , 6 Or 7 . Bryan's in t

In [5]:
len(sample_data)

2131

In [6]:
import random
random.shuffle(sample_data)
train_data = sample_data[:1600]
test_data = sample_data[1600:]

In [7]:
len(test_data)

531

In [8]:
training_data=[]
for i in range (0,1600):
    training_data.append({"class":train_data[i].split('\t')[1].split('\n')[0],"sentence":train_data[i].split('\t')[0]})

In [9]:
training_data

[{'class': 'event',
  'sentence': "I love Coldplay's Billie Jean cover . I hope they release an official acoustic version or something ."},
 {'class': 'trifle', 'sentence': 'I need to go to the shooting range .'},
 {'class': 'career',
  'sentence': 'i wanna be a TV extra any ideas on how i can do this ?'},
 {'class': 'trifle',
  'sentence': 'I want a love like Noah from The Notebook*sigh ***'},
 {'class': 'non-intent',
  'sentence': 'I should be able to knock out this project today .'},
 {'class': 'trifle', 'sentence': 'I need a new hobby ....'},
 {'class': 'food', 'sentence': 'I need something cold ... yumm a popsicle'},
 {'class': 'trifle',
  'sentence': "I wanna wreck my stockings in some jukebox dive . It's a Joni Mitchell kind of day today ."},
 {'class': 'food',
  'sentence': "I want papa johns so bad if jamon doesn't hook us up I don't like jamon ."},
 {'class': 'non-intent',
  'sentence': "I just don't believe you . And if you are lying then I'm DONE . That's all I have to say 

In [10]:
corpus_words = {}
class_words = {}
# turn a list into a set (of unique items) and then a list again (this removes duplicates)
classes = list(set([a['class'] for a in training_data]))
for c in classes:
    # prepare a list of words within each class
    class_words[c] = []

# loop through each sentence in our training data
for data in training_data:
    # tokenize each sentence into words
    for word in nltk.word_tokenize(data['sentence']):
        # ignore a some things
        if word not in ["?", "'s"]:
            # stem and lowercase each word
            stemmed_word = stemmer.stem(word.lower())
            # have we not seen this word already?
            if stemmed_word not in corpus_words:
                corpus_words[stemmed_word] = 1
            else:
                corpus_words[stemmed_word] += 1

            # add the word to our words in class list
            class_words[data['class']].extend([stemmed_word])

# we now have each stemmed word and the number of occurances of the word in our training corpus (the word's commonality)
print ("Corpus words and counts: %s \n" % corpus_words)
# also we have all words in each class
print ("Class words: %s" % class_words)

Corpus words and counts: {'i': 2667, 'lov': 76, 'coldplay': 2, 'bil': 3, 'jean': 4, 'cov': 2, '.': 1311, 'hop': 46, 'they': 41, 'releas': 1, 'an': 53, 'off': 27, 'acoust': 2, 'vert': 3, 'or': 45, 'someth': 39, 'nee': 358, 'to': 1041, 'go': 224, 'the': 520, 'shoot': 6, 'rang': 1, 'wan': 169, 'na': 207, 'be': 181, 'a': 650, 'tv': 5, 'extr': 1, 'any': 26, 'idea': 10, 'on': 199, 'how': 30, 'can': 72, 'do': 187, 'thi': 105, 'want': 311, 'lik': 94, 'noah': 1, 'from': 45, 'notebook*sigh': 1, '***': 1, 'should': 86, 'abl': 10, 'knock': 2, 'out': 63, 'project': 3, 'today': 63, 'new': 77, 'hobby': 1, '...': 408, 'cold': 9, 'yum': 4, 'popsic': 1, 'wreck': 1, 'my': 327, 'stock': 1, 'in': 228, 'som': 103, 'jukebox': 1, 'div': 1, 'it': 264, 'jon': 2, 'mitchel': 1, 'kind': 9, 'of': 165, 'day': 73, 'pap': 4, 'john': 2, 'so': 118, 'bad': 32, 'if': 41, 'jamon': 2, 'doe': 10, "n't": 174, 'hook': 3, 'us': 38, 'up': 71, 'just': 159, 'believ': 15, 'you': 158, 'and': 348, 'ar': 30, 'lying': 2, 'then': 25, "'

In [11]:
def calculate_class_score_commonality(sentence, class_name, show_details=True):
    score = 0
    # tokenize each word in our new sentence
    
    for word in nltk.word_tokenize(sentence):
        # check to see if the stem of the word is in any of our classes
        if stemmer.stem(word.lower()) in class_words[class_name]:
            # treat each word with relative w
            score += (1 / corpus_words[stemmer.stem(word.lower())])

            if show_details:
                print ("   match: %s (%s)" % (stemmer.stem(word.lower()), 1 / corpus_words[stemmer.stem(word.lower())]))
    return score

In [12]:
def classify(sentence):
    high_class = None
    high_score = 0
    # loop through our classes
    for c in class_words.keys():
        # calculate score of sentence for each class
        score = calculate_class_score_commonality(sentence, c, show_details=False)
        # keep track of highest score
        if score > high_score:
            high_class = c
            high_score = score

    return high_class, high_score

In [13]:
classify("I think I tweet too much ... I really don't wana hit 2000 tweets b4 september lol ... Sometimes I don't even know I'm twitting lol")

('non-intent', 2.353509840392069)

In [14]:
classify(test_data[502])

('goods', 0.05962411417833817)

In [15]:
3

3

In [16]:
testing_data=[]
for i in range (0,531):
    testing_data.append({"class":test_data[i].split('\t')[1].split('\n')[0],"sentence":test_data[i].split('\t')[0]})

In [17]:
testing_data

[{'class': 'event',
  'sentence': 'i need to get a shower and then go to band practice . any suggestions for songs ppl ? Also , i been listening to the kinks , such a good band .'},
 {'class': 'food',
  'sentence': 'I want Starbucks !! Maybe that would help wake me up !!!'},
 {'class': 'food',
  'sentence': "I need me some KFC =) I'm hungry be back in a little bit"},
 {'class': 'goods',
  'sentence': 'i may need help with this digital switchover . do i need a new antenna ???'},
 {'class': 'non-intent',
  'sentence': 'I hate the question " soo what\'s new with you " idk why I just don\'t like it'},
 {'class': 'non-intent',
  'sentence': 'I can’t see you though I think you face in this direction .'},
 {'class': 'event',
  'sentence': "I just want to run into the wide , comforting arms of ventrilo and forget about my classes . But I suppose that's one of the reasons I'm here ."},
 {'class': 'non-intent',
  'sentence': 'i think i just made of ass out of myself !'},
 {'class': 'food', 'sent

In [18]:
classify("I really really want a sunday with extra chocolate")[0]

'career'

In [19]:
cor=0
fal=0
for i in range (0,531):
    if(classify(testing_data[i]["sentence"])[0]==testing_data[i]["class"]):
        cor+=1
    else:
        fal+=1
print(cor)
print(fal)

182
349


In [20]:
cor=0
fal=0
for i in range (0,1600):
    if(classify(training_data[i]["sentence"])[0]==training_data[i]["class"]):
        cor+=1
    else:
        fal+=1
print(cor)
print(fal)

1405
195
